# A Hubble Source Catalog (HSC) Use Case

- [Example 6][1]: Using the Discovery Portal to study the Red Sequence in a Galaxy Cluster
  - (The Red Sequence in the Galaxy Cluster Abell 2390)


  [1]: https://archive.stsci.edu/hst/hsc/help/use_case_6_v1.html

<span style="color:red;">GOAL</span>: This tutorial shows you how to use the [MAST Discovery Portal][1] to create a Color-Magnitude diagram of extended sources.

<span style="color:red;">SCIENCE CASE</span>: The science case is to isolate the red sequence - a color-magnitude relation for elliptical and lenticular galaxies in clusters of galaxies - in a cluster well observed by HST (i.e. Abell 2390 at z=0.2; see [Gladders and Yee 2000, AJ, 120, 2148][2]). Aperture corrections and extinction corrections are also performed.
Other potential use cases could include testing cluster evolution via the change in the slope of the red sequence, and identifying other clusters with pronounced red sequences.


  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: https://ui.adsabs.harvard.edu/abs/2000AJ....120.2148G/abstract

In [ ]:
import astropy
from astropy.coordinates import SkyCoord
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

from PIL import Image
from io import BytesIO

from astropy.table import Table, join
from astropy.io import ascii

# set width for pprint
astropy.conf.max_width = 150

In [ ]:
# import hsc catalog search utilites from local py file
from search_utils import *

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

## <span style="color:red;">Step 1</span> - Go to the [MAST Discovery Portal][1].

Use the pull down menu under Select Collection to choose the HSC (<span style="color:blue;">blue</span>). Enter the name of the cluster (or if you prefer the coordinates) and search radius (i.e. Abell 2390 r=0.2d) in the Search box (<span style="color:green;">green</span>). Perform the search by just hitting a carriage return. The results are displayed in the List of Objects, while the AstroView window shows the objects against the DSS image. The left column is a series of Filters that can be used to refine the data selected.

![redesequence_1][2]


  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: screenshots/redsequence_1.png

### Use SkyCoord name resolver to get the position of Adell 2390

In [ ]:
target = 'Abell 2390'
coord_adell2390 = SkyCoord.from_name(target)

ra_adell2390 = coord_adell2390.ra.degree
dec_adell2390 = coord_adell2390.dec.degree
print(f'ra: {ra_adell2390}\ndec: {dec_adell2390}')

### Cone search with HSC API

In [ ]:
radius = 0.2 # degrees
cone = hcvcone(ra_adell2390, dec_adell2390, radius, table="summary")
tab = ascii.read(cone)
df = tab.to_pandas()
df.head()

In [ ]:
# number of observations
len(df)

In [ ]:
g = sns.relplot(data=df, x='MatchRA', y='MatchDec', color='purple', height=10, 
                hue='TargetName', hue_order=sorted(df.TargetName.unique()), alpha=.5)
for ax in g.axes.ravel():
    ax.invert_xaxis()

## <span style="color:red;">Step 2</span> - Refine the sample.

The HSC includes both compact (point) and extended sources, as well as a few residual cosmic rays and image artifacts. Since we are looking for galaxies (i.e. extended sources), we want to remove as many point sources as possible. Scroll down the Filters section (<span style="color:blue;">blue</span>) to the CI (Concentration Index) area and set the lower limit to be 1.3 by either moving the slider or typing in the value (<span style="color:green;">green</span>). Note that about 1/3 of the objects have been rejected (<span style="color:orange;">orange</span>).

![redsequence_2][1]


  [1]: screenshots/redsequence_2.png

### Label and filter the DataFrame by `'CI'`

- Point sources such as stars have CI ~ 1 (**compact**), and objects with CI significantly greater than 1 are bigger than point sources, meaning they are likely to be galaxies (**extended**).

In [ ]:
df['label'] = np.where(df.CI.gt(1.3), 'extended', 'compact')

In [ ]:
df.head()

In [ ]:
dfe = df[df.label.eq('extended')]

In [ ]:
len(dfe)

In [ ]:
g = sns.relplot(data=dfe, x='MatchRA', y='MatchDec', height=10, hue='TargetName',
                hue_order=sorted(df.TargetName.unique()), alpha=.5)
for ax in g.axes.ravel():
    ax.invert_xaxis()

## <span style="color:red;">Step 3</span> - Determine the HSC magnitudes corrections.

To make our Color-Magnitude diagram, we need to correct the magnitude values for the V (W2_F555W = WFPC2 F555W) and I (W2_F814W = WFPC2 F814W) to infinite aperture magnitudes for better comparison, and correct for Galactic extinction along the line of sight.

Approximate aperture corrections can be taken from the [aperture corrections table][1], or estimated from the encircled energy curves provided by the instrument teams. For both filters, the aperture corrections are 0.17 mag, or:

W2_F555W_Inf = W2_F555W - 0.17
W2_F814W_Inf = W2_F814W - 0.17.

Extinction correction is best estimated using an absorption calculator, like [Doug's Excellent Absorption Law Calculator][2]. The HSC table includes a column labeled Extinction, which is the Schlegel, Finkbeiner, & Davis 1998 ([ApJ, 500, 525, 1998][3]) E(B-V)=0.114 map value at that position. Using the absorption law calculator, with R_V = 3.1, and A_V = E(B-V) x R_V = 0.353, and assuming target wavelength equal to the pivot wavelength for each filter, or:

W_V = 0.535 (um)
W_I = 0.820 (um).

We get the following extinction corrections for each filter:

W2_F555W_Ext = W2_F555W - 0.36
W2_F814W_Ext = W2_F814W - 0.20,

and arrive at the final corrections for each filter:

W2_F555W_Cor = W2_F555W - 0.53
W2_F814W_Cor = W2_F814W - 0.37.


  [1]: https://archive.stsci.edu/hst/hsc/help/FAQ/aperture_corrections.txt
  [2]: http://dogwood.physics.mcmaster.ca/Acurve.html
  [3]: http://adsabs.harvard.edu/abs/1998ApJ...500..525S

## <span style="color:red;">Step 4</span> - Create new Magnitude and Color columns

To correct the magnitudes, we will create new columns and apply the corrections determined above. Click on the ![icon][1] icon (<span style="color:blue;">blue</span>). In the popup, select the first value to be "W2_F555W", the operation to be "x+c", and the constant to be "-0.53"; enter the name as VC (V corrected).

![redsequence_3][2]

Do the same thing to make the corrected W2_F814W (I) magnitude. Finally, we need to create the VC-IC color by selecting "VC", "-", "IC", and "VC-IC".


  [1]: screenshots/new_column.png
  [2]: screenshots/redsequence_3.png

In [ ]:
df['VC'] = df['W2_F555W'].sub(0.53)
df['IC'] = df['W2_F814W'].sub(0.37)

df['VC-IC'] = df['VC'].sub(df['IC'])

df[['VC', 'IC', 'VC-IC']].tail()

In [ ]:
# drop nan rows
df = df.dropna(subset=['VC', 'IC', 'VC-IC'])

display(df.head())
len(df)

## <span style="color:red;">Step 5</span> - Make the Color-Magnitude Diagram

Select the ![chart][1] icon (<span style="color:blue;">blue</span>). Set X = IC and Y = VC-IC (<span style="color:green;">green</span>). Click on the Update Series button (<span style="color:orange;">orange</span>) to plot the diagram.

![redsequence_4][2]

There is clear sequence of sources near the middle of the diagram running with a negative slope. A linear regression fit feature may be added in future Discovery Portal versions, but for now one can estimate the slope of the sequence from the grid, or by moving the cursor within the plot, and find it consistent with m=-0.04. This is the slope of the red sequence expected from clusters in the z=0.2 range, as shown in Gladders & Yee 2000, Figure 2.

A more exact value of the slope could be obtained by downloading the table using the ![export][3] icon, and employing whatever program you use to perform linear regression fits (e.g., using Python and 3-sigma clipping, we get a value of -0.042 +/- 0.007, which is in good agreement with the value of -0.037 +/- 0.0042 from [Gladders et. al. 1998, ApJ, 501, 571][4]).

The HSC figure is shifted slightly relative to the Gladders et al. plot since the HSC uses ABMAG and Gladders uses STMAG magnitudes. The conversion equations are

W2_F555W(STmag) = W2_F555W(AB) - 0.108
W2_F814W(STmag) = W2_F814W(AB) + 0.824.

obtainable using stsdas.synphot, see also the [WFPC2 Photometry FAQ][5].

![redsequence_5][6]


  [1]: screenshots/chart.png
  [2]: screenshots/redsequence_4.png
  [3]: screenshots/export.png
  [4]: http://adsabs.harvard.edu/abs/1998ApJ...501..571G
  [5]: http://www.stsci.edu/hst/wfpc2/wfpc2_faq/wfpc2_phot_faq.html
  [6]: screenshots/redsequence_5.png

In [ ]:
g = sns.relplot(data=df, x='IC', y='VC-IC', height=10, hue='label')

### Regression Line

#### `sklearn` regression line

In [ ]:
model = LinearRegression()

In [ ]:
# selecte only the extended data
data = df[df.label.eq('extended')]

In [ ]:
_ = model.fit(data['IC'].values.reshape((-1, 1)), data['VC-IC'].values)

In [ ]:
model.intercept_

In [ ]:
model.coef_

In [ ]:
x = np.linspace(18, 25, 1000)

In [ ]:
y = model.predict(x.reshape((-1, 1)))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(data=df, x='IC', y='VC-IC', hue='label')
sns.lineplot(x=x, y=y, color='tab:orange', ax=ax, label='extended regression line')

sns.move_legend(ax, bbox_to_anchor=(1, 0.5), loc='center left', title='label', frameon=False)
ax.spines[['top', 'right']].set_visible(False)

#### `scipy` regression line

In [ ]:
# calculate regression line with scipy
linregress(data['IC'].values, data['VC-IC'].values)

#### seaborn regression plot

In [ ]:
# plot the regression line directly with lmplot
g = sns.lmplot(data=data, x='IC', y='VC-IC', height=10, hue='label', palette=sns.color_palette("tab10")[1:])

In [ ]:
# plot regression line for extended and add compact points
fig, ax = plt.subplots(figsize=(12, 10))
sns.regplot(data=data, x='IC', y='VC-IC', color='tab:orange', scatter_kws=dict(s=15), ax=ax, label='extended')
sns.scatterplot(data=df[df.label.eq('compact')], x='IC', y='VC-IC', color='tab:blue', ax=ax, label='compact')

sns.move_legend(ax, bbox_to_anchor=(1, 0.5), loc='center left', title='label', frameon=False)
ax.spines[['top', 'right']].set_visible(False)